In [ ]:
! pip install transformers datasets
! pip install tqdm
!pip install gliner

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split
#import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder

In [12]:
import pandas as pd

csv_file_1 = './data/embedding_training_data/Round1_train.csv'
csv_file_2 = './data/embedding_training_data/Round3_train.csv'
csv_file_3 = './data/embedding_training_data/2T_train.csv'
csv_file_4 = './data/embedding_training_data/HT2_train.csv'

# Read the CSV files
df1 = pd.read_csv(csv_file_1)
df2 = pd.read_csv(csv_file_2)
df3 = pd.read_csv(csv_file_3)
df4 = pd.read_csv(csv_file_4)

dataset = pd.concat([df1, df2, df3, df4], axis=0)


In [13]:
from gliner import GLiNER
from collections import Counter
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

# Assuming model is loaded
model = GLiNER.from_pretrained("urchade/gliner_base")

# 384 THE TOKEN LIMITS

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/792M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/gliner/model.py:568: FutureWarning: You are using `torch.load` with `wei

In [ ]:
# Define new labels and the mapping to the original labels
new_labels = [
    "person", "organization", "location", "others", "movie", "videogames", "separator"
]

# Define a mapping from new labels to original labels
label_mapping = {
    "person": "person",
    "organization": "organization",
    "location": "location",
    "movie": "others",
    "others": "others",
    "videogames": "others",
    "date": "others",
    "galaxy": "others"
}

# Define the separator
SEPARATOR = " ||| "

df = []
batch_size = 30

# Process the dataset in chunks of 1000 rows
for start in tqdm(range(0, dataset.shape[0], batch_size), desc="Processing Batches"):
    batch = dataset.iloc[start:start + batch_size]
    #print(batch.iloc[-1])
    # Prepare the text for the entire batch with a separator between each entry
    batch_text = SEPARATOR.join(batch['text'] + ' - ' + batch['desc'])  # Using SEPARATOR variable

    # Predict entities for the entire batch
    batch_entities = model.predict_entities(batch_text, new_labels, threshold=0.05)
    #print(batch_entities[-1])

    try:
        ner_type = []
        text = batch_text[:batch_text.find(SEPARATOR)].split(" - ")[0]    
        index = batch[batch['text'] == text].index[0]
        target = batch.iloc[index]['label']
        desc = batch_text[:batch_text.find(SEPARATOR)].split(" - ")[1]
    except:
        # this cover the case of index 
        index = 0
        target = batch.iloc[index]['label']
        desc = batch_text[:batch_text.find(SEPARATOR)].split(" - ")[1]

    for el in batch_entities:

        if batch_text.find(el['text']) <= batch_text.find(SEPARATOR):
            ner_type.append(el['label'])
            
        else:
            counter = Counter(ner_type)
            try:
                # Get the most common NER type
                most_common_element, element_count = counter.most_common(1)[0]

                #print(f"{el['text']}-{el['label']} - most_common_element: {most_common_element}")
                # Append the new row to the list
                new_row = {
                    'text': text,
                    'target': target,
                    'prediction': label_mapping[most_common_element]
                }
                df.append(new_row)
                #print(f"{text}-->{new_row}")
                
                ner_type = []  
                batch_text = batch_text[batch_text.find(SEPARATOR)+len(SEPARATOR):]
                text = batch_text[:batch_text.find(SEPARATOR)].split(" - ")[0]  
                index = batch[batch['text'] == text].index[0]
                target = batch.iloc[index]['label']
                #desc = batch.iloc[index-start]['desc']
                desc = batch_text[:batch_text.find(SEPARATOR)].split(" - ")[1]
                
                #print(f"target: {target}, prediction: {most_common_element} --> {text} - {desc}")
                #print(f"{el['text']}-{text}")
                if el['text'] == text:
                    ner_type.append(el['label'])
            except Exception as e:
                # Print the exception type and message
                batch_text = batch_text[batch_text.find(SEPARATOR)+len(SEPARATOR):]
                continue
            
# Convert the list to a DataFrame for further analysis or saving
result_df = pd.DataFrame(df)


Processing Batches:   1%|▏         | 14/935 [00:13<14:20,  1.07it/s]/opt/conda/lib/python3.11/site-packages/gliner/data_processing/processor.py:206: UserWarning: Sentence of length 390 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
Processing Batches:   3%|▎         | 30/935 [00:30<17:05,  1.13s/it]

In [191]:
df = pd.DataFrame(df)


In [192]:
df

,text,target,prediction
0,need for speed: carbon,OTHERS,others
1,alex rodriguez,PERS,person
2,lex luger,PERS,person
3,crash bash,OTHERS,others
4,the hurt locker,OTHERS,others
...,...,...,...
478,KIC 9832227,OTHERS,location
479,Pegnitz,OTHERS,location
480,"Coach ill, Calgary",OTHERS,location
481,Call e,OTHERS,person


In [19]:
# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(df)

# Save the DataFrame to a CSV file
df.to_csv('./data/embedding_training_data/1_predictions.csv', index=False)

print("DataFrame saved to predictions.csv")

DataFrame saved to predictions.csv


In [5]:
csv_file_1 = 'data/embedding_training_data/1_predictions.csv'
csv_file_2 = 'data/embedding_training_data/2_predictions.csv'
# Read the CSV files
df1 = pd.read_csv(csv_file_1)
df2 = pd.read_csv(csv_file_2)

df = pd.concat([df1, df2], axis=0)


In [7]:
df[:50]

,text,target,prediction
0,need for speed: carbon,OTHERS,others
1,alex rodriguez,OTHERS,person
2,school of rock,OTHERS,organization
3,lex luger,OTHERS,person
4,crash bash,OTHERS,others
5,the hurt locker,OTHERS,others
6,lion,OTHERS,others
7,fear and loathing in las vegas,OTHERS,others
8,kidsty pike,LOC,location
9,randy johnson,OTHERS,person


In [11]:
## COMPUTE HOW MANY CLASSES ARE MISSING

# Mapping between labels
mapping = {
    "OTHERS": "others",
    "PERS": "person",
    "LOC": "location",
    "ORG": "organization"
}

# Apply the mapping to the target column
df['mapped_target'] = df['target'].map(mapping)

# Initialize a dictionary to count wrong predictions for each class
wrong_predictions_count = {}
total_class_count = {}

# Iterate through each row in the dataframe
for index, row in df.iterrows():
    total_class_count[row['mapped_target']] = total_class_count.get(row['mapped_target'], 0) + 1

    # Check if the mapped target is not equal to the prediction
    if row['mapped_target'] != row['prediction']:
        print(f"{row['text']}: {row['mapped_target']} != {row['prediction']}")
        # Increment the count for the corresponding class
        if row['mapped_target'] in wrong_predictions_count:
            wrong_predictions_count[row['mapped_target']] += 1
        else:
            wrong_predictions_count[row['mapped_target']] = 1



relative_percentage = {
    cls: (wrong_predictions_count.get(cls, 0) / total_class_count[cls]) * 100
    for cls in total_class_count
}

# Print the relative percentage of wrong predictions for each class
print("Relative percentage of wrong predictions for each class:", relative_percentage)

alex rodriguez: others != person
school of rock: others != organization
lex luger: others != person
randy johnson: others != person
timeshift: others != location
greenland: others != location
new taiwan dollar: others != location
pope linus: others != person
mark langston: others != person
life as a house: others != location
ice harvest: others != location
dead poets society: others != organization
pope innocent i: others != person
archives of internal medicine: others != organization
st helena pounds: others != location
singstar rocks!: others != person
world of warcraft: wrath of the lich king: others != location
abraham the syrian: others != location
leymus innovatus: others != person
ziziphus jujuba: others != person
us virgin islands: others != location
chrysobalanus icaco: others != person
ncaa football 2004: others != organization
jackie robinson: others != person
frasier: others != person
city lights: others != location
eddie guerrero: others != person
pope anicetus: others != 

In [ ]:
total_class_count

In [ ]:
balanced_dataset['label'].value_counts()

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm

# Sample dataset
data = dataset

# Extract descriptions and targets
descriptions = data["desc"].tolist()
texts = data["text"].tolist()
targets = data["label"].tolist()

# Map targets to integers
target_map = {"PERS": 0, "ORG": 1, "LOC": 2, "OTHERS": 3}
y = [target_map[target] for target in targets]

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')
model = BertModel.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')

# Generate embeddings
def get_embeddings(texts):
    embeddings = []
    for text in tqdm(texts, desc="Generating Embeddings", unit="text"):
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        # Use the CLS token embeddings (first token) as the sentence representation
        embedding = outputs.last_hidden_state[:, 0, :].numpy()
        embeddings.append(embedding)
    return np.vstack(embeddings)

# Get embeddings for both fields
desc_embeddings = get_embeddings(descriptions)
text_embeddings = get_embeddings(texts)



In [ ]:
import numpy as np
import torch

# Save the concatenated and mean embeddings
np.save('concatenated_embeddings.npy', concatenated_embeddings)
np.save('mean_embeddings.npy', mean_embeddings)

# Save the BERT model and tokenizer
model.save_pretrained('bert_model')
tokenizer.save_pretrained('bert_tokenizer')



In [ ]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel

# Load the concatenated and mean embeddings
concatenated_embeddings = np.load('concatenated_embeddings.npy')
mean_embeddings = np.load('mean_embeddings.npy')

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert_tokenizer')
model = BertModel.from_pretrained('bert_model')


In [ ]:
## LOGISTIC REGRESSION MODEL


concatenated_embeddings = np.concatenate((desc_embeddings, text_embeddings), axis=1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(concatenated_embeddings, y, test_size=0.2, random_state=42)

# Train a logistic regression classifier on embeddings
clf = LogisticRegression(max_iter=1000, verbose=1)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')


In [ ]:
### NN MODEL
import torch
import torch.nn as nn
import torch.optim as optim

X_train, X_test, y_train, y_test = train_test_split(concatenated_embeddings, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Define the neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 300)  # Input to hidden layer
        self.relu = nn.ReLU()  # ReLU activation
        self.fc2 = nn.Linear(300, 4)  # Hidden layer to output layer (4 classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
model_NN = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_NN.parameters(), lr=0.001)

# Training the model
n_epochs = 10
for epoch in range(n_epochs):
    model_NN.train()
    optimizer.zero_grad()
    outputs = model_NN(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Print the loss every epoch
    print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model_NN.eval()
with torch.no_grad():
    y_pred_tensor = model_NN(X_test_tensor)
    y_pred = torch.argmax(y_pred_tensor, dim=1).numpy()

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

In [ ]:
def predict_label(text, desc):
    # Generate embedding for the new text
    desc_embedding = get_embeddings([desc])
    text_embedding = get_embeddings([text])

    # Concatenate the embeddings
    concatenated_embedding = np.concatenate((desc_embedding, text_embedding), axis=1)

    # Predict using the trained classifier
    pred = clf.predict(concatenated_embedding)

    # Map the predicted integer label back to the original category
    label = reverse_target_map[pred[0]]
    return label

def predict_label_NN(text, desc):
    # Generate embedding for the new text and description
    text_embedding = get_embeddings([text])
    desc_embedding = get_embeddings([desc])

    # Concatenate the embeddings
    concatenated_embedding = np.concatenate((desc_embedding, text_embedding), axis=1)
    mean_embedding = (desc_embedding + text_embedding) / 2

    # Convert the numpy array to a PyTorch tensor
    input_tensor = torch.tensor(concatenated_embedding, dtype=torch.float32)

    # Predict using the trained neural network model
    model_NN.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        output = model_NN(input_tensor)
        pred = torch.argmax(output, dim=1).item()

    # Map the predicted integer label back to the original category
    label = reverse_target_map[pred]
    return label


# Example usage: Predicting a new description
reverse_target_map = {v: k for k, v in target_map.items()}  # For decoding predictions


# List of short descriptions to test
text = [
    "Statue of Liberty",
    "Grand Canyon",
    "Great Wall of China",
    "Sahara Desert",
    "Sydney Opera House",
    "Taj Mahal",
    "Niagara Falls",
    "Amazon River",
    "Mount Kilimanjaro",
    "Yellowstone National Park",
    "Burj Khalifa",
    "Times Square",
    "Albert Einstein",
    "Marie Curie",
    "Leonardo da Vinci",
    "Google",
    "NASA",
    "United Nations",
    "Eiffel Tower",
    "Mount Everest",
    "Amazon Rainforest",
    "Mona Lisa",
    "Photosynthesis",
    "Theory of Evolution"
]

descriptions = [
     "a colossal neoclassical sculpture on Liberty Island in New York City, USA.",
    "a steep-sided canyon carved by the Colorado River in Arizona, USA.",
    "an ancient series of walls and fortifications in northern China.",
    "the world's largest hot desert, located in North Africa.",
    "a multi-venue performing arts center in Sydney, Australia.",
    "a white marble mausoleum located in Agra, India.",
    "a group of three waterfalls at the border of Canada and the USA.",
    "the largest river by discharge volume in the world, located in South America.",
    "a dormant volcano in Tanzania, and the highest mountain in Africa.",
    "a large national park known for its geothermal features, located in the USA.",
    "the tallest building in the world, located in Dubai, UAE.",
    "a major commercial intersection and tourist destination in New York City, USA.",
    " German-born theoretical physicist, developer of the theory of relativity.",
    "physicist and chemist, pioneer in radioactivity research.",
    "Italian polymath, painter, engineer, and scientist.",
    " American technology company specializing in Internet-related services.",
    " U.S. government agency for space exploration and research.",
    " international organization founded in 1945 for global peace and security.",
    "wrought-iron tower in Paris, France.",
    "Earth's highest mountain, located in the Himalayas.",
    "large tropical rainforest in South America.",
    "portrait painting by Leonardo da Vinci, housed in the Louvre.",
    "process by which plants convert light into chemical energy.",
    "scientific theory by Darwin on natural selection."
]

# Predict labels for each description
for t, d in zip(text, descriptions):
    predicted_label = predict_label(t, d)
    print(f'{predicted_label} --> text: {t}, Description: "{d}"\n')

In [ ]:
# Save the BERT model and tokenizer to a directory
model_save_path = './bert_model'
tokenizer_save_path = './bert_tokenizer'

# Save model
model.save_pretrained(model_save_path)

# Save tokenizer
tokenizer.save_pretrained(tokenizer_save_path)




In [ ]:
from transformers import BertTokenizer, BertModel

# Define the paths where the model and tokenizer are saved
model_save_path = './bert_model'
tokenizer_save_path = './bert_tokenizer'

# Load the tokenizer
#tokenizer = BertTokenizer.from_pretrained(tokenizer_save_path)

# Load the model
model = BertModel.from_pretrained(model_save_path)


In [ ]:

from collections import Counter

category_counts = Counter(df1['label'])

# Display the counts for each category
for category, count in category_counts.items():
    print(f'{category}: {count}')

In [ ]:
###############
# Direct Fine-Tuning:
################


import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric

# Load dataset from CSV files
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)  # 4 classes

# Tokenization function that uses the 'description' column
def tokenize(batch):
    return tokenizer(batch['description'], padding='max_length', truncation=True, max_length=512)

# Apply the tokenization to the dataset
train_dataset = dataset['train'].map(tokenize, batched=True)
test_dataset = dataset['test'].map(tokenize, batched=True)

# Prepare the dataset for PyTorch (Torch tensors)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',             # Output directory
    num_train_epochs=3,                 # Number of training epochs
    per_device_train_batch_size=16,     # Batch size for training
    per_device_eval_batch_size=16,      # Batch size for evaluation
    warmup_steps=500,                   # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                  # Strength of weight decay
    evaluation_strategy="epoch",        # Evaluate at the end of each epoch
    logging_dir='./logs',               # Directory for storing logs
    logging_steps=10,                   # Log every 10 steps
)

# Load evaluation metric
metric = load_metric("accuracy")

# Define the compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define Trainer instance
trainer = Trainer(
    model=model,                         # The pre-trained model
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=test_dataset,           # Evaluation dataset
    compute_metrics=compute_metrics,     # Function to compute evaluation metrics
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./bert-finetuned')

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Test accuracy: {eval_results['eval_accuracy']:.4f}")



In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('./bert-finetuned')

# Example new data
new_texts = ["paris", "michael"]

# Tokenize the new data
def tokenize_new_data(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

# Tokenize
new_data = tokenize_new_data(new_texts)

# Set the model to evaluation mode
model.eval()

# Use the model to get predictions
with torch.no_grad():
    outputs = model(**new_data)
    logits = outputs.logits

# Convert logits to probabilities
probabilities = torch.nn.functional.softmax(logits, dim=-1)

# Get predicted labels
predictions = torch.argmax(probabilities, dim=-1)

# Define label mappings
id2label = {0: 'PERS', 1: 'ORG', 2: 'LOC', 3: 'OTHERS'}

# Convert predictions to labels
predicted_labels = [id2label[pred.item()] for pred in predictions]

# Print results
for text, label in zip(new_texts, predicted_labels):
    print(f"Text: {text}\nPredicted Label: {label}\n")


In [ ]:
def get_embedding(text):
    # Tokenize the input text and get the input IDs
    inputs = tokenizer(text, return_tensors='pt')
    # Get the hidden states from the model
    outputs = model(**inputs)
    # Use the mean of the hidden states as the embedding
    embedding = torch.mean(outputs.last_hidden_state, dim=1)
    return embedding

In [ ]:
label_text = "science fiction writer"
qid_embedding = "movie"

# Get embeddings
label_embedding = get_embedding(label_text)


# Calculate cosine similarity
distance = cosine(qid_embedding.detach().numpy(), label_embedding.detach().numpy())
print(f"Cosine distance: {distance}")

